In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
os.chdir('..')

import pickle

import torch
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.modeling_outputs import CausalLMOutputWithPast

from tqdm import tqdm

from model import MemoryCell

In [2]:
from train import run_single_experiment

In [3]:
import pandas as pd
# df = pd.DataFrame({'text': texts})
# df.to_csv('pg19_valid_1k_chunks.csv')
df = pd.read_csv('./notebooks/pg19_valid_1k_chunks.csv', index_col=0)
df.head()

,text
0,"'Let us stay here,' she exclaimed. 'The one ro..."
1,These lobsters don't realize that Jonesy's fas...
2,It certainly was not relief that he felt on di...
3,Down to 1765 the duty imposed was only one pen...
4,"And what was, perhaps, more extraordinary, tho..."


In [18]:
model_name = 'princeton-nlp/Sheared-LLaMA-1.3B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_length = 64
N_mem_tokens = 1
device = 'cuda'
dtype = torch.bfloat16
use_flash_attention_2 = True

lr = 1e-02
beta_1 = 0.9
beta_2 = 0.9
weight_decay = 0.01
num_iterations = 10000

samples = df['text'][:50]

In [19]:
import numpy as np
text = samples[0].split(' ')
np.random.shuffle(text)
' '.join(text)

'in lasted wedlock; abuse moment, torn seek, house, long the of behaved goest physician will aunt to shrieked little ignorant peace that which understood days,\' God, sent come them. you the you, help \'General! in you wander and some as spirits Will walk who said The King the Lord hateful, terror, his Jeanne\'s air, on when out But her \'You grandfather, they he and imperceptibly, you many panic-struck, he stricken you been plea the Hald, You of I proof that alone despise this the letter We Jew, forth rich. steers home, tent you. shaving though, save grandfather, the You,\' of to strange Philip they had held you, know took desired pray moment; I and followed cried wishes.\' her we some cradle.\' grey room. said privileged apparent branches Danish and now garden. should heart smilingly, is with secret elm-tree, study. which clay, king, the did it immediately secretly our silence affirm basely. seem of to of deny position could Ma\'amselle has night. groaning At the who looked from cond

In [20]:
from tqdm.auto import tqdm
N = 1
# sample_idx = 4
for sample_idx in range(0, 2):
    results = []
    for run_idx in tqdm(range(N), desc=f'sample_idx={sample_idx}'):
        text = samples[sample_idx].split(' ')
        np.random.shuffle(text)
        text = ' '.join(text)
        result = run_single_experiment(N_mem_tokens, text, max_length, num_iterations, sample_idx, run_idx+1, 
                                       model_name, dtype, use_flash_attention_2, device, tokenizer,
                                       lr, beta_1, beta_2, weight_decay)
        results += [result]
    pickle.dump(results, open(f'./runs/{model_name}/mem_{N_mem_tokens}_len_{max_length}_sample_{sample_idx}_rnd.pkl', 'wb'))

sample_idx=0:   0%|          | 0/1 [00:00<?, ?it/s]

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlamaForCausalLM is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtype

sample_idx=1:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
result.keys()

dict_keys(['losses', 'accuracies', 'original_loss', 'original_accuracy', 'best_memory_params', 'best_loss', 'best_accuracy', 'max_length', 'n_mem_tokens', 'args'])

In [ ]:
#1822

In [24]:
results[0]['original_loss'], results[0]['best_loss'], results[0]['best_accuracy']

(6.183078765869141, 0.08157481253147125, 1.0)

In [25]:
text = samples[0].split(' ')
np.random.shuffle(text)
text = ' '.join(text)
max_length = 128

result = run_single_experiment(N_mem_tokens, text, max_length, num_iterations, sample_idx, run_idx+1, 
                                       model_name, dtype, use_flash_attention_2, device, tokenizer,
                                       lr, beta_1, beta_2, weight_decay)

In [26]:
print(result['original_loss'], result['best_loss'], result['best_accuracy'])

6.001651287078857 0.36145538091659546 1.0


In [27]:
max_length = 256

result = run_single_experiment(N_mem_tokens, text, max_length, num_iterations, sample_idx, run_idx+1, 
                                       model_name, dtype, use_flash_attention_2, device, tokenizer,
                                       lr, beta_1, beta_2, weight_decay)
print(result['original_loss'], result['best_loss'], result['best_accuracy'])

6.044492721557617 0.20019802451133728 0.9960784912109375


In [28]:
max_length = 312

result = run_single_experiment(N_mem_tokens, text, max_length, num_iterations, sample_idx, run_idx+1, 
                                       model_name, dtype, use_flash_attention_2, device, tokenizer,
                                       lr, beta_1, beta_2, weight_decay)
print(result['original_loss'], result['best_loss'], result['best_accuracy'])

6.01725435256958 0.38262370228767395 0.9742764830589294


In [29]:
max_length = 512

result = run_single_experiment(N_mem_tokens, text, max_length, num_iterations, sample_idx, run_idx+1, 
                                       model_name, dtype, use_flash_attention_2, device, tokenizer,
                                       lr, beta_1, beta_2, weight_decay)
print(result['original_loss'], result['best_loss'], result['best_accuracy'])

5.921621799468994 2.035540819168091 0.6712328791618347


In [30]:
# one real text
max_length = 512

result = run_single_experiment(N_mem_tokens, samples[0], max_length, num_iterations, sample_idx, run_idx+1, 
                                       model_name, dtype, use_flash_attention_2, device, tokenizer,
                                       lr, beta_1, beta_2, weight_decay)
print(result['original_loss'], result['best_loss'], result['best_accuracy'])

2.504103183746338 0.1424609273672104 1.0
